In [1]:
!pip install wurlitzer
!pip install -q --upgrade keras-cv tensorflow
!pip install -q --upgrade keras

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires tensorflow~=2.15.0, but you have tensorflow 2.16.1 which is incompatible.
tensorflow-text 2.15.0 requires tensorflow<2.16,>=2.15.0; platform_machine != "arm64" or platform_system != "Darwin", but you have tensorflow 2.16.1 which is incompatible.
tf-keras 2.15.1 requires tensorflow<2.16,>=2.15, but you have tensorflow 2.16.1 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires tensorflow~=2.15.0, but you have tensorflow 2.16.1 which is incompatible.
tensorflow-text 2.15.0 requires tensorflow<2.16,>=2.15.0; platform_machine != "arm64" or platform_system != "Darwin", but you have tensorflow

In [2]:
import tensorflow as tf
import keras
import os
import pandas as pd
from tensorflow.keras.preprocessing import image
from sklearn.model_selection import train_test_split

In [3]:
os.environ["KERAS_BACKEND"] = "tensorflow"  # Or "jax" or "torch"!

In [4]:
input_size = 224
num_frames = 32
batch_size = 1  # Adjust based on your strategy
AUTOTUNE = tf.data.experimental.AUTOTUNE

def read_frames_from_folder(folder_path):
    folder_path = folder_path.numpy().decode('utf-8')
    # Get list of all frame files
    frame_files = sorted([os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.jpg')])
    
    # Take the last num_frames frames if there are more frames
    if len(frame_files) > num_frames:
        frame_files = frame_files[-num_frames:]
    
    frames = []
    for frame_file in frame_files:
        img = image.load_img(frame_file, target_size=(input_size, input_size))
        img_array = image.img_to_array(img)
        frames.append(img_array)
    
    frames = tf.convert_to_tensor(frames, dtype=tf.float32)

    # If fewer than num_frames, pad with zeros
    if len(frames) < num_frames:
        padding = tf.zeros((num_frames - len(frames), input_size, input_size, 3), dtype=tf.float32)
        frames = tf.concat([frames, padding], axis=0)
    return frames

def load_video_from_folder(file_path):
    video = tf.py_function(func=read_frames_from_folder, inp=[file_path], Tout=tf.float32)
    video.set_shape([num_frames, input_size, input_size, 3])
    return video

def create_test_dataset(folder_path, file_names):
    file_paths = file_names.apply(lambda x: os.path.join(folder_path, x)).values
    dataset = tf.data.Dataset.from_tensor_slices(file_paths)
    dataset = dataset.map(load_video_from_folder, num_parallel_calls=AUTOTUNE)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(AUTOTUNE)
    return dataset


In [5]:
keras_model = keras.models.load_model('/kaggle/input/mipr-ava-zero-models/best_model.keras')

In [6]:
# Load the sample submission file
submission_df = pd.read_csv('/kaggle/input/2nd-ava-challenge-ieee-mipr-2024/2nd_AVA_Dataset_2ed/sample_submission.csv')

# Filter test file names for road and freeway
road_test_files = submission_df[submission_df['file_name'].str.contains('road')]
freeway_test_files = submission_df[submission_df['file_name'].str.contains('freeway')]

# Paths to test data
freeway_test_path = '/kaggle/input/2nd-ava-challenge-ieee-mipr-2024/2nd_AVA_Dataset_2ed/freeway/test'
road_test_path = '/kaggle/input/2nd-ava-challenge-ieee-mipr-2024/2nd_AVA_Dataset_2ed/road/test'

# Create test datasets
road_test_dataset = create_test_dataset(road_test_path, road_test_files['file_name'])
freeway_test_dataset = create_test_dataset(freeway_test_path, freeway_test_files['file_name'])


In [7]:
# Run inference
def run_inference(dataset, model):
    predictions = []
    for videos in dataset:
        preds = model.predict(videos)
        predictions.extend(preds)
    return predictions

road_predictions = run_inference(road_test_dataset, keras_model)
freeway_predictions = run_inference(freeway_test_dataset, keras_model)

I0000 00:00:1719113879.380022      77 service.cc:145] XLA service 0x79d738001400 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1719113879.380075      77 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0


1/1 ━━━━━━━━━━━━━━━━━━━━ 23s 23s/step


I0000 00:00:1719113885.834379      77 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'input_add_reduce_fusion_2', 4 bytes spill stores, 4 bytes spill loads

I0000 00:00:1719113885.853901      77 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━

In [8]:
import numpy as np

# Softmax function to convert logits to probabilities
def softmax(logits):
    exp_logits = np.exp(logits)
    return exp_logits / np.sum(exp_logits, axis=-1, keepdims=True)

# Convert logits to probabilities using softmax
road_probabilities = [softmax(pred) for pred in road_predictions]
freeway_probabilities = [softmax(pred) for pred in freeway_predictions]

# Extract the probability of the positive class (class 1)
road_positive_probs = [prob[1] for prob in road_probabilities]
freeway_positive_probs = [prob[1] for prob in freeway_probabilities]

In [9]:
# Convert probabilities to binary risk labels
# road_binary_labels = [1 if prob >= 0.5 else 0 for prob in road_positive_probs]
# freeway_binary_labels = [1 if prob >= 0.5 else 0 for prob in freeway_positive_probs]

# Update submission dataframe
submission_df.loc[submission_df['file_name'].str.contains('road'), 'risk'] = road_positive_probs
submission_df.loc[submission_df['file_name'].str.contains('freeway'), 'risk'] = freeway_positive_probs

# Save the updated submission file
submission_df.to_csv('submission.csv', index=False)

/tmp/ipykernel_23/2702639105.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.4707707, 0.1259051, 0.3102384, 0.25008795, 0.14911194, 0.027377134, 0.047815476, 0.9509566, 0.078959994, 0.055059455, 0.71671957, 0.10751497, 0.057105474, 0.56998223, 0.29617503, 0.030530157, 0.5242995, 0.044715058, 0.0036746254, 0.07663289, 0.39093393, 0.054099377, 0.034573194, 0.16446112, 0.1701304, 0.22147495, 0.005943684, 0.0956102, 0.25533387, 0.29995087, 0.18266694, 0.011598811, 0.108081825, 0.065355495, 0.012218216, 0.44177303, 0.31772125, 0.15904279, 0.37738326, 0.42072764, 0.34710225, 0.39029178, 0.5444139, 0.20178035, 0.22831567, 0.03405409, 0.017858462, 0.1292447, 0.18627544, 0.29301882, 0.112266324, 0.1722476, 0.7321478, 0.6563965, 0.06443679, 0.07267776, 0.027988445, 0.32612506, 0.03395908, 0.14052328, 0.056101892, 0.0011305462, 0.08879208, 0.65322864, 0.40073773, 0.0108239, 0.09441757, 0.48538563, 0.17443247